In [3]:
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd

leagues = {
    'premier_league': list(range(1982, 1983)),
    'ligue_1': list(range(1989, 2000)),
    'serie_a': list(range(1994, 2000)),
    'la_liga': list(range(1993, 2000))
}

options = webdriver.ChromeOptions()
options.add_argument('headless')

driver = webdriver.Chrome(options=options)

for league, years in leagues.items():
    for year in years:
        if league == 'premier_league':
            url = f'https://fbref.com/en/comps/9/{year}-{year+1}/{year}-{year+1}-Premier-League-Stats'
        elif league == 'ligue_1':
            url = f'https://fbref.com/en/comps/13/{year}-{year+1}/{year}-{year+1}-Ligue-1-Stats'
        elif league == 'serie_a':
            url = f'https://fbref.com/en/comps/11/{year}-{year+1}/{year}-{year+1}-Serie-A-Stats'
        elif league == 'la_liga':
            url = f'https://fbref.com/en/comps/12/{year}-{year+1}/{year}-{year+1}-La-Liga-Stats'
        driver.get(url)
        try:
            tables = driver.find_elements(By.TAG_NAME, "table")

            dfs = []
            for table in tables:
                html = table.get_attribute("outerHTML")
                df = pd.read_html(html)[0]
                dfs.append(df)

            result = pd.concat(dfs)

            result.to_csv(f'{league}_season_standings_{year}.csv', index=False)


        except:
            print("not possible")
            pass

print(f"Scraped {len(result)} tables from {len(url)} pages.")

# Close the browser session
driver.quit()

Scraped 44 tables from 69 pages.


In [1]:
### extract all soccer data

from dotenv import load_dotenv
load_dotenv(override=True)
import os
from os.path import join
import time

from web_scraping import get_soccer_leagues_ids, get_soccer_league_season_standings, get_soccer_teams_url, get_player_season_stats

current_season = 2022
soccer_league_ids = get_soccer_leagues_ids()

for year in range(1982,2000):
    print(year)
    try:
        for soccer_league_id in soccer_league_ids:
            if soccer_league_id["league_name"] == "Fußball-Bundesliga" or soccer_league_id["league_name"] == "Big 5 European Leagues Combined":
                continue

            # get player stats
            df = get_player_season_stats(year, current_season, soccer_league_id["league_id"], soccer_league_id["league_stats_name"])
            df.to_csv(join(os.environ["SportsTables"], "soccer", f"{soccer_league_id['league_name'].replace(' ','-')}_player_stats_{year}.csv"), index=False)

            time.sleep(3)
    except:
        print("not possible")
        continue


1982
Scraped 66 tables from 22 pages.
not possible
